# Submissions for The Kaggle Playground Series Season 4 Episode 4
https://www.kaggle.com/competitions/playground-series-s4e4/overview

Evaluation metric is Root mean squared logarithmic error. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

: 

In [2]:
TEST = pd.read_csv("test.csv")
TEST_id = TEST[["id"]]
TEST = TEST.drop(columns=["id"])
TRAIN = pd.read_csv("train.csv").drop(columns = ["id"])

TEST_d = pd.get_dummies(TEST)
TRAIN_d = pd.get_dummies(TRAIN)
X = TRAIN_d.drop(columns=["Rings"])
y = TRAIN_d["Rings"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=.8)

In [5]:
def calc_RMSLE(actual, predicted):
    actual = actual.values
    n = len(actual)
    df = pd.DataFrame({"actual":actual, "predicted":predicted})
    df["val"] = (np.log(1+df.predicted) - np.log(1+df.actual))**2
    RMSLE = df.val.sum()/n
    return RMSLE

In [65]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
preds = rf.predict(X_test)

In [76]:
print(calc_RMSLE(y_test, preds))

18123
438.1585662411894
0.024176933523213012


## Basic Random Forest Submission
Score: .15538

In [45]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X, y)
preds = rf.predict(TEST_d)
SUBMISSION["Rings"] = np.round(preds)

c:\Users\mattc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[10.21  9.82 10.14 ... 12.   12.45  8.52]


In [52]:
SUBMISSION.to_csv("./submissions/Apr26Submission1.csv", index=False)

## Linear Regression Submission:
Score: 0.16707

In [61]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X,y)
preds = lr.predict(TEST_d)
SUBMISSION["Rings"] = np.round(preds)
SUBMISSION
SUBMISSION.loc[SUBMISSION.Rings < 0,"Rings"] = 0

In [62]:
SUBMISSION.to_csv("./submissions/Apr26Submission2.csv", index=False)

# Normalization
score: .15535

In [3]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import root_mean_squared_log_error

ss = StandardScaler()
ss_TRAIN = ss.fit_transform(X)
ss_TEST = ss.fit_transform(TEST_d)

ss_TRAIN = pd.DataFrame(columns=X.columns, data=ss_TRAIN)

ss_TEST = pd.DataFrame(columns=TEST_d.columns, data=ss_TEST)

Xt, Xv, yt, yv = train_test_split(ss_TRAIN, y, train_size=.8)

In [12]:
kf = KFold(shuffle=True)
rmsle = []
for train_index, test_index in kf.split(ss_TRAIN):
    train, test = ss_TRAIN.iloc[train_index], ss_TRAIN.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    rf = RandomForestRegressor(n_estimators=1000)
    rf.fit(train, y_train)
    preds = np.round(rf.predict(test))
    rmsle.append(root_mean_squared_log_error(y_test, preds))

print(sum(rmsle)/5)

0.15651395637714327


In [4]:
SUB = pd.read_csv("./sample_submission.csv")
SUB

,id,Rings
0,90615,10
1,90616,10
2,90617,10
3,90618,10
4,90619,10
...,...,...
60406,151021,10
60407,151022,10
60408,151023,10
60409,151024,10


In [16]:
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(ss_TRAIN, y)
SUB["Rings"] = np.round(rf.predict(ss_TEST))
SUB.to_csv("./submissions/Apr28Submission1.csv", index=False)

# Trying with Classifier
Score: .16989

In [17]:
rf = RandomForestClassifier(n_estimators=2000)
rf.fit(ss_TRAIN, y)
SUB["Rings"] = rf.predict(ss_TEST)
SUB.to_csv("./submissions/Apr28Submission2.csv", index=False)

# LightGBM Model
score: .15201

In [19]:
from lightgbm import LGBMRegressor


model = LGBMRegressor(metric="rmsle", n_estimators=800)

model.fit(ss_TRAIN, y)
preds = model.predict(ss_TEST)
SUB["Rings"] = np.round(preds)
SUB.to_csv("./submissions/Apr28Submission3.csv", index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002529 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1341
[LightGBM] [Info] Number of data points in the train set: 90615, number of used features: 10
[LightGBM] [Info] Start training from score 9.696794


# XGBoost Model
score: .15857

In [20]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=800, objective="reg:squaredlogerror")

model.fit(ss_TRAIN, y)
preds = model.predict(ss_TEST)

SUB["Rings"] = np.round(preds)
SUB.to_csv("./submissions/Apr28Submission4.csv", index=False)

# LightGBM Hyperparam Tuning
score: .15193

In [12]:
import optuna
from numba import jit
import lightgbm as lgb
from lightgbm import LGBMRegressor

def objective(trial):
    params = {
        "verbose":"-1",
        "metric":"rmsle",
        "device":"gpu",
        "learning_rate":trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        "num_leaves":trial.suggest_int("num_leaves", 15, 100),
        "max_depth":trial.suggest_int("max_depth",10,1000),
        "estimators":trial.suggest_int("estimators", 50, 5000),
        "min_child_samples":trial.suggest_int("min_child_samples", 2, 200),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0)
    }

    model = LGBMRegressor(**params)
    model.fit(Xt, yt)
    preds = np.round(model.predict(Xv))
    return root_mean_squared_log_error(yv, preds)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)
best_params = study.best_params
print(best_params)

bm = LGBMRegressor(**best_params)
bm.fit(ss_TRAIN, y)
SUB["Rings"] = np.round(bm.predict(ss_TEST))

SUB.to_csv("./submissions/Apr28Submission5.csv", index=False)

[I 2024-04-28 20:47:01,101] A new study created in memory with name: no-name-c0d1584a-d172-493d-a4be-e388ccc52ea2
C:\Users\mattc\AppData\Local\Temp\ipykernel_13600\492465691.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":trial.suggest_loguniform('learning_rate', 0.005, 0.5),
C:\Users\mattc\AppData\Local\Temp\ipykernel_13600\492465691.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
C:\Users\mattc\AppData\Local\Temp\ipykernel_13600\492465691.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/o

{'learning_rate': 0.12105939823236904, 'num_leaves': 90, 'max_depth': 261, 'estimators': 4817, 'min_child_samples': 193, 'reg_alpha': 0.5234592337359771, 'reg_lambda': 0.010939940111873195}
[LightGBM] [Warning] Unknown parameter: estimators
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: estimators
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000687 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1341
[LightGBM] [Info] Number of data points in the train set: 90615, number of used features: 10
[LightGBM] [Info] Start training from score 9.696794
[LightGBM] [Warning] Unknown parameter: estimators


In [13]:
tm = LGBMRegressor(**best_params)
tm.fit(Xt, yt)
preds = np.round(tm.predict(Xv))
root_mean_squared_log_error(yv, preds)

[LightGBM] [Warning] Unknown parameter: estimators
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: estimators
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002146 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1338
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 10
[LightGBM] [Info] Start training from score 9.687414
[LightGBM] [Warning] Unknown parameter: estimators


0.15258631814702162

In [ ]:
import optuna
from numba import jit
import lightgbm as lgb
from lightgbm import LGBMRegressor

def objective(trial):
    params = {
        "verbose":"-1",
        "metric":"rmsle",
        "device":"gpu",
        "learning_rate":trial.suggest_float('learning_rate', 0.005, 0.5, log=True),
        "num_leaves":trial.suggest_int("num_leaves", 15, 100),
        "max_depth":trial.suggest_int("max_depth",10,1000),
        "estimators":trial.suggest_int("estimators", 50, 5000),
        "min_child_samples":trial.suggest_int("min_child_samples", 2, 200),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)
    }

    model = LGBMRegressor(**params)
    model.fit(Xt, yt)
    preds = np.round(model.predict(Xv))
    return root_mean_squared_log_error(yv, preds)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)
best_params = study.best_params
print(best_params)

bm = LGBMRegressor(**best_params)
bm.fit(ss_TRAIN, y)
SUB["Rings"] = np.round(bm.predict(ss_TEST))

SUB.to_csv("./submissions/Apr28Submission5.csv", index=False)